In [1]:
import random
from copy import deepcopy

########### Low level functions ##########
## Get random name from list
## Requires list not be empty
def get_random_name(list):
    index = random.randrange(0, len(list))
    return list[index]

## Remove name from list
## If name not in list, does nothing
def remove_name_from_list(list, remove_name):
    shortened_list = deepcopy(list)
    if remove_name in shortened_list:
        shortened_list.remove(remove_name)
    return shortened_list

## Check if assignments work
## If given empty list, returns True
def check_assignments(list_of_dicts):
    assignment_dict_1 = list_of_dicts[0]
    for key in assignment_dict_1:
        for i in range(1, len(list_of_dicts)):
            if (assignment_dict_1[key] == list_of_dicts[i][key]):
                return False
    return True    

## Put Assignments together
## Requires list_of_dicts not empty
def put_assignments_together(list_of_dicts):
    ## Setup final dict and first assignment
    final_assignment_dict = {}
    assignment_dict_1 = list_of_dicts[0]
    
    ## Combine the recipients of each buyer
    for key in assignment_dict_1:
        temp_recip_list = [assignment_dict_1[key]]
        for i in range(1, len(list_of_dicts)):
            temp_recip_list.append(list_of_dicts[i][key])
        final_assignment_dict[key] = temp_recip_list
    return final_assignment_dict
    
## Add new assignment dict to list of dicts
## Without any repeats
def add_new_assignment(list_of_dicts, full_name_list):
    while(True):
        temp_ass_dict = {}
        get_name_assignment(full_name_list, temp_ass_dict)
        temp_list_of_dicts = deepcopy(list_of_dicts)
        temp_list_of_dicts.append(temp_ass_dict)
        if (check_assignments(temp_list_of_dicts)):
            return temp_list_of_dicts    

########### High level functions ##########
## Try to assign each buyer one recipient
def try_get_name_assignment(full_name_list, assignment_dict):
    recipient_list = deepcopy(full_name_list)
    
    ## Assign recipients
    for i in range(0, len(full_name_list)):
        ## Get buyer
        buyer = full_name_list[i]

        ## If recipient list only has buyer left, then failed
        shortened_list = remove_name_from_list(recipient_list, buyer)
        if (len(shortened_list) == 0):
            assignment_dict = {}
            return False
        
        ## Get random recipient, excluding buyer
        recipient = get_random_name(shortened_list)

        ## Remove recipient from recipient list
        recipient_list.remove(recipient)

        ## Add buyer-> recipient pair
        assignment_dict[buyer] = recipient
    return True

## Assign each buyer one recipient
def get_name_assignment(full_name_list, assignment_dict):
    success = False
    while (not success):
        success = try_get_name_assignment(full_name_list, assignment_dict)
        
## Assign each buyer two recipients
## Requires recipients_per_buyer > 0
def get_name_assignments(full_name_list, recipients_per_buyer):
    ## Make Assignment 1
    assignment_dict_1 = {}
    get_name_assignment(full_name_list, assignment_dict_1)

    ## Make other assignments
    list_of_dicts = [assignment_dict_1]
    while (len(list_of_dicts) < recipients_per_buyer):
        list_of_dicts = add_new_assignment(list_of_dicts, full_name_list)
        while(True):
            temp_ass_dict = {}
            get_name_assignment(full_name_list, temp_ass_dict)
            temp_list_of_dicts = deepcopy(list_of_dicts)
            temp_list_of_dicts.append(temp_ass_dict)
            if (check_assignments(temp_list_of_dicts)):
                list_of_dicts = deepcopy(temp_list_of_dicts)
                break

    ## Put Assignments together
    final_assignment_dict = put_assignments_together(list_of_dicts)
    
    return final_assignment_dict

In [2]:
import smtplib

########### Sending Email ##########
def send_email(TO, SUBJECT, BODY, USER = "learningpython640@gmail.com", PASSWORD = "ilearnpython"):

    email_text = """\
From: %s  
To: %s  
Subject: %s

%s
""" % (USER, ", ".join(TO), SUBJECT, BODY)

    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(USER, PASSWORD)
        server.sendmail(USER, TO, email_text)
        server.close()

        print("Email sent")
    except:
        print("Something went wrong")

In [3]:
########## Make tree ##########
def make_tree(n,s):
    tree = ""
    for i in range(n):
        for a in range(n-i):
            tree = tree + " "
        tree = tree + "["
        ## Added reduce length to half the width (didn't look good on email)
        reduce_length = 0
        for l in range(i<<1):
            if i==n-1:
                reduce_length = reduce_length + 1
                if reduce_length % 2 == 0:
                    tree = tree + "_"
            else:
                reduce_length = reduce_length + 1
                if reduce_length % 2 == 0:
                    tree = tree + "~"
        tree = tree + "]\n"
    for o in range(s):
        for i in range(n):
            tree = tree + " "
        tree = tree + "[]\n"
    return tree

In [4]:
########## Output Dictionary Content into String ##########
def show_dict(dict):
    dict_output = ""
    for key in dict:
        dict_output = dict_output + "{0}: {1}\n".format(key, dict[key])
    return dict_output

In [5]:
########### Program ###########

## Setup emails and names
EMAIL_ASSIGNMENT = {"Tyler": "tylergwlum@gmail.com", "Nancy": "nancygylum@gmail.com", "Jenny": "jennlums@gmail.com", "Sandy": "sandygmlum@gmail.com", "Kent": "sio.kent@gmail.com", "Peter": "wang.peter91@gmail.com"}
NICKNAMES = {"Tyler": "Turdeep", "Nancy": "Naan", "Jenny": "Jenitals", "Sandy": "Sandpaper", "Peter": "Bowflexer", "Kent": "Yucatan"}
NAME_LIST = list(EMAIL_ASSIGNMENT.keys())
recipients_per_buyer = 2

## Get assignment
full_assignment_list = get_name_assignments(NAME_LIST, recipients_per_buyer)

## Send emails
for key in full_assignment_list:
    recipients = full_assignment_list[key]
    
    ## Get nicknames
    buyer_nickname = NICKNAMES[key]
    recipient_nickname_1 = NICKNAMES[recipients[0]]
    recipient_nickname_2 = NICKNAMES[recipients[1]]
    
    ## Write email body
    email_body = "Hi {0}! You will be buying a lovely present for: {1} and {2}! :)".format(buyer_nickname, recipient_nickname_1, recipient_nickname_2)
     
    ## From statement
    email_body = email_body + "\n\n-Tyler Bot"
    
    ## Nickname explanation
    email_body = email_body + "\n\nNICKNAMES:\n{0}".format(show_dict(NICKNAMES))
    
    ## Add tree
    email_body = email_body + "\n\n" + make_tree(11, 3)
   
    ## Subject
    subject = "Sibling Secret Santa Draw Trial Run 1"

    print(email_body)
    #send_email([EMAIL_ASSIGNMENT[key]], subject, email_body)
    
## Send email of actual draw to hidden email
#send_email(["learningpython640@gmail.com"], "Sibling Secret Santa Real List", "{0}".format(full_assignment_list))

Hi Turdeep! You will be buying a lovely present for: Bowflexer and Sandpaper! :)

-Tyler Bot

NICKNAMES:
Tyler: Turdeep
Nancy: Naan
Jenny: Jenitals
Sandy: Sandpaper
Peter: Bowflexer
Kent: Yucatan


           []
          [~]
         [~~]
        [~~~]
       [~~~~]
      [~~~~~]
     [~~~~~~]
    [~~~~~~~]
   [~~~~~~~~]
  [~~~~~~~~~]
 [__________]
           []
           []
           []

Hi Naan! You will be buying a lovely present for: Jenitals and Yucatan! :)

-Tyler Bot

NICKNAMES:
Tyler: Turdeep
Nancy: Naan
Jenny: Jenitals
Sandy: Sandpaper
Peter: Bowflexer
Kent: Yucatan


           []
          [~]
         [~~]
        [~~~]
       [~~~~]
      [~~~~~]
     [~~~~~~]
    [~~~~~~~]
   [~~~~~~~~]
  [~~~~~~~~~]
 [__________]
           []
           []
           []

Hi Jenitals! You will be buying a lovely present for: Naan and Bowflexer! :)

-Tyler Bot

NICKNAMES:
Tyler: Turdeep
Nancy: Naan
Jenny: Jenitals
Sandy: Sandpaper
Peter: Bowflexer
Kent: Yucatan


           []
        